In [ ]:
import pandas as pd
import numpy as np
from math import radians
import folium
import requests
from scipy.optimize import minimize
from time import time
from tqdm.notebook import tqdm

<h1>Nelder-Mead Simplex optimization</h1>

<h2>Reading dataset</h2>
For the purpose of the experiment, the results are only shown for a particular cluster.

In [ ]:
df = pd.read_csv('./Olist_data/clusters_new_features.csv', index_col= 0)
df.head(5)

In [ ]:
c1 = df[df.CLUSTER_hdbscan == 86]

<h2>Plotting cluster on a map</h2>

In [ ]:
mapf = folium.Map(
    location= [c1.geolocation_lat.mean(),c1.geolocation_lng.mean()],
    zoom_start = 10.5,
    tiles= 'OpenStreetMap',
    height= 550
)

circles= c1.apply(
    lambda row: folium.CircleMarker(
        location= [row.geolocation_lat, row.geolocation_lng],
        radius= 1,
        popup= ""+str(row.geolocation_lat)+", "+str(row.geolocation_lng)+"\n"+str(row.CLUSTER_hdbscan),
        color= 'darkcyan',
        fill= True,
        fill_color= 'darkcyan',
        fill_opacity=0.5
    ).add_to(mapf),
    axis= 1
)

rect= folium.Rectangle(
    bounds= [(c1.geolocation_lat.min(),c1.geolocation_lng.min()),(c1.geolocation_lat.max(), c1.geolocation_lng.max())]
).add_to(mapf)

cluster= folium.CircleMarker(
    location= [c1.centroid_lat.values[0],c1.centroid_lng.values[0]],
    radius= 5,
    color= 'tomato',
    fill= True,
    fill_color= 'tomato',
    fill_opacity=0.5
).add_to(mapf)


mapf

<h2>Feature engineering functions</h2>
Functions to calculate Haversine distance and retrieve road distance and road duration using Google Maps API

In [ ]:
maps_api_key = 'ENTER YOUR API KEY'
endpoint = 'https://maps.googleapis.com/maps/api/directions/json?'

def get_distance_duration(origin, destination, mode = 'driving'):
    origin_str = f'{origin[0]},{origin[1]}'
    destination_str = f'{destination[0]},{destination[1]}'
    request_url = f'{endpoint}origin={origin_str}&destination={destination_str}&mode={mode}&key={maps_api_key}'
    response = requests.get(request_url)
    data = response.json()
    
    if data['status'] == 'OK':
        road_distance_meters = data['routes'][0]['legs'][0]['distance']['value']
        road_duration_seconds = data['routes'][0]['legs'][0]['duration']['value']

        return road_distance_meters/1000, road_duration_seconds/3600
        
    else:
        print('Error: Unable to calculate road distance and duration.')
        return None

def haversine(latlon1, latlon2):
    lat1, lon1 = latlon1
    lat2, lon2 = latlon2
    R = 6371000  # radius of Earth in meters
    phi_1 = radians(lat1)
    phi_2 = radians(lat2)

    delta_phi = radians(lat2 - lat1)
    delta_lambda = radians(lon2 - lon1)

    a = (np.sin(delta_phi / 2) ** 2 +
         np.cos(phi_1) * np.cos(phi_2) * np.sin(delta_lambda / 2) ** 2)

    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    meters = R * c  
    return meters/1000 # output distance in km

<h2>Nelder-Mead simplex optimizer class</h2>
Functions wrapped in a class to apply the optimizer to custom dataset.

In [ ]:
class NelderMeadOptimizer():
    PARAM_BOUNDS = (0,10)
    
    def __init__(self, lat_bounds, long_bounds, customer_locations, max_iter):
        self.LAT_BOUNDS = lat_bounds
        self.LON_BOUNDS = long_bounds
        self.customer_locations = customer_locations
        self.max_iter = max_iter
        
        
    def initialize_guess(self):
        vector = np.random.uniform(
            [self.LAT_BOUNDS[0], self.LON_BOUNDS[0]] + [NelderMeadOptimizer.PARAM_BOUNDS[0]] * 3,
            [self.LAT_BOUNDS[1], self.LON_BOUNDS[1]] + [NelderMeadOptimizer.PARAM_BOUNDS[1]] * 3,
        )
        return vector


    def update_features(self, new_location, customer_locations):
        dist = []
        dur = []
        hav = []
        origin = (new_location[0], new_location[1])
    
        for loc in customer_locations:
            data = get_distance_duration(origin, tuple(loc))
            
            if data:
                hav.append(haversine(origin, tuple(loc)))
                dist.append(data[0])
                dur.append(data[1])
    
        return pd.Series(hav), pd.Series(dist), pd.Series(dur)


    def cost_function(self, position, lambda_reg=0.01, epsilon=1e-6):
        latitude, longitude, alpha, beta, gamma = position
        haversine_distances, road_distances, road_durations = self.update_features(position, self.customer_locations)
    
        cost = (1 / len(haversine_distances)) * sum(
            alpha * haversine_distances +
            beta * road_distances +
            gamma * road_durations
        ) + lambda_reg * (alpha**2 + beta**2 + gamma**2)
    
        log_penalty = -np.sum(np.log(np.array([alpha, beta, gamma]) + epsilon))
        cost = cost + log_penalty
    
        return cost

    def fit_train(self):
        options = {
            'maxiter': self.max_iter
        }
        self.initial_guess = self.initialize_guess()
        result = minimize(self.cost_function, self.initial_guess, method='Nelder-Mead', options= options)
        return result

In [ ]:
nmo = NelderMeadOptimizer(
    lat_bounds= (c1.geolocation_lat.min(), c1.geolocation_lat.max()),
    long_bounds= (c1.geolocation_lng.min(), c1.geolocation_lng.max()),
    customer_locations= np.c_[np.array(c1.geolocation_lat), np.array(c1.geolocation_lng)],
    max_iter= 5
)

result = nmo.fit_train()
result

<h2>Cost and performance vs multiple iterations</h2>

In [ ]:
params = np.arange(2,25,2)
best_loc = []
best_score = []
path = []
performance = []

for iter in tqdm(params):
    nmo = NelderMeadOptimizer(
        lat_bounds= (c1.geolocation_lat.min(), c1.geolocation_lat.max()),
        long_bounds= (c1.geolocation_lng.min(), c1.geolocation_lng.max()),
        customer_locations= np.c_[np.array(c1.geolocation_lat), np.array(c1.geolocation_lng)],
        max_iter= iter
    )
    start = time()
    result = nmo.fit_train()
    stop = time()
    
    best_loc.append(result.x) 
    best_score.append(result.fun) 
    performance.append(stop - start)

<h2>Plotting new warehouse location on map</h2>

In [ ]:
mapf = folium.Map(
    location= [c1.geolocation_lat.mean(),c1.geolocation_lng.mean()],
    zoom_start = 11.5,
    tiles= 'OpenStreetMap',
    height= 650
)

circles= c1.apply(
    lambda row: folium.CircleMarker(
        location= [row.geolocation_lat, row.geolocation_lng],
        radius= 1,
        popup= ""+str(row.geolocation_lat)+", "+str(row.geolocation_lng)+"\n"+str(row.CLUSTER_hdbscan),
        color= 'darkcyan',
        fill= True,
        fill_color= 'darkcyan',
        fill_opacity=0.5
    ).add_to(mapf),
    axis= 1
)

rect= folium.Rectangle(
    bounds= [(c1.geolocation_lat.min(),c1.geolocation_lng.min()),(c1.geolocation_lat.max(), c1.geolocation_lng.max())]
).add_to(mapf)

cluster1= folium.CircleMarker(
    location= [c1.centroid_lat.values[0],c1.centroid_lng.values[0]],
    radius= 4052.7209882681345/120,
    color= 'tomato',
    fill= True,
    fill_color= 'tomato',
    fill_opacity=0.5
).add_to(mapf)

cluster2= folium.CircleMarker(
    location= [-16.676760, -49.298581],
    radius= 109.855445/10,
    color= 'magenta',
    fill= True,
    fill_color= 'magenta',
    fill_opacity=0.5
).add_to(mapf)

mapf

<h2>Bootstrapping algorithm</h2>

In [ ]:
score = []
position = []
perf= []

def bootstrap_results(num_boot=100):
    for i in tqdm(range(num_boot)):
        sub_sample = c1.sample(n= len(c1), replace= True, random_state= 12)
        nmo = NelderMeadOptimizer(
            lat_bounds= (c1.geolocation_lat.min(), c1.geolocation_lat.max()),
            long_bounds= (c1.geolocation_lng.min(), c1.geolocation_lng.max()),
            customer_locations= np.c_[np.array(sub_sample.geolocation_lat), np.array(sub_sample.geolocation_lng)],
            max_iter= 2
        )
        start = time()
        result = nmo.fit_train()
        stop = time()

        position.append(result.x)
        score.append(result.fun)
        perf.append(stop-start)

    return score, position, perf

bootstrap_results(num_boot= 20)